## Textfile_from_WRFoutput_pythonic
- to convert wrf output into text file for the value of a specified lat long
- Using the script [WrfncXnj.py](http://www.meteo.unican.es/wiki/cordexwrf/SoftwareTools/WrfncXnj), convert wrf output in cf abiding nc file
- the steps are as follows
- cp wrfout_d04* /home/hoopoe/wrfncxnj-0.1_r2120/
- to get the list of available files in a directory
```python
import os
a=[]
for file in os.listdir("/home/hoopoe/wrfncxnj-0.1_r2120/"):
    if file.startswith("wrfout_d04"):
        a.append(file)
```
- to run a external python script from another python script, 
```python
import subprocess
a= wrfout_d04_2014-06-11_00:00:00
subprocess.call("python wrfncxnj.py --split-variables --output-pattern=ZZZG7wrfout_[varcf].nc -v 'T2,Q2,TDPS,MSLP,RAIN' "+a,shell=True)
```
- to get unique output from the python execution
```python
import subprocess
a= wrfout_d04_2014-06-11_00:00:00
subprocess.call("python wrfncxnj.py --split-variables --output-pattern="+a+"_[varcf].nc -v 'T2,Q2,TDPS,MSLP,RAIN' "+a,shell=True)
```
- to make a loop to get the file name from list and run the script with each file name
```python
import subprocess
import os
a=[]
for file in os.listdir("/home/hoopoe/wrfncxnj-0.1_r2120/"):
    if file.startswith("wrfout_d04"):
        a.append(file)
for fl in a:
    subprocess.call("python wrfncxnj.py --split-variables --output-pattern="+fl+"_[varcf].nc -v 'T2,Q2,TDPS,MSLP,RAIN' "+fl,shell=True)
```
- this makes nc files with five variables for each wrf output
- to make variable values out of nc file and make a csv file, using kd tree algortih fro [here]
```python
import numpy as np
import netCDF4
from math import pi
from numpy import cos, sin
from scipy.spatial import cKDTree
import os
import csv
class Kdtree_fast(object):
    def __init__(self, ncfile, latvarname, lonvarname):
        self.ncfile = ncfile
        self.latvar = self.ncfile.variables[latvarname]
        self.lonvar = self.ncfile.variables[lonvarname]        
        # Read latitude and longitude from file into numpy arrays
        rad_factor = pi/180.0 # for trignometry, need angles in radians
        self.latvals = self.latvar[:] * rad_factor
        self.lonvals = self.lonvar[:] * rad_factor
        self.shape = self.latvals.shape
        clat,clon = cos(self.latvals),cos(self.lonvals)
        slat,slon = sin(self.latvals),sin(self.lonvals)
        clat_clon = clat*clon
        clat_slon = clat*slon
        triples = zip(np.ravel(clat*clon), np.ravel(clat*slon), np.ravel(slat))
        self.kdt = cKDTree(triples)
    def query(self,lat0,lon0):
        rad_factor = pi/180.0 
        lat0_rad = lat0 * rad_factor
        lon0_rad = lon0 * rad_factor
        clat0,clon0 = cos(lat0_rad),cos(lon0_rad)
        slat0,slon0 = sin(lat0_rad),sin(lon0_rad)
        dist_sq_min, minindex_1d = self.kdt.query([clat0*clon0,clat0*slon0,slat0])
        iy_min, ix_min = np.unravel_index(minindex_1d, self.shape)
        return iy_min,ix_min
a=[]
for file in os.listdir("/home/hoopoe/wrfncxnj-0.1_r2120/"):
    if file.endswith("huss.nc"):
        a.append(file)
writer=csv.writer(open("/home/hoopoe/wrfncxnj-0.1_r2120/huss.csv",'wb'))
for nc in a:
	ncfile = netCDF4.Dataset(nc, 'r')
	ns = Kdtree_fast(ncfile,'lat','lon')
	iy,ix = ns.query(8.4875, 76.9525)
	tas = ncfile.variables['huss']
	print 'Closest lat lon:', ns.latvar[iy,ix], ns.lonvar[iy,ix]
	print 'Temperature:', tas[0, 0, iy, ix], tas.units
        print nc,' ',tas[0, 0, iy, ix]
        b= nc,tas[0, 0, iy, ix]
        writer.writerow(b)
	ncfile.close()
```